In [1]:
import brian2
from brian2 import mV, ms, nS, mS, nF, Hz

import matplotlib.pyplot as plt

In [2]:
dt = 0.001*ms
inv_dt = 1./dt
update = inv_dt

N_e = 800
N_ir = 150
N_if = 50
N_all = N_e + N_ir + N_if

t_init = 500*inv_dt
t_pre = 0*inv_dt
t_stim = 500*inv_dt
t_post = 100*inv_dt
t_all = t_init + t_pre + t_stim + t_post

In [3]:
con_t, con_r, con_p = 0, 0, 0
import random
print(random.getstate())

(3, (4079360547, 1638891505, 3342417467, 2305098635, 895831760, 3151052183, 418279589, 3524445948, 3226248084, 3890044862, 2194971507, 4079291438, 2480359334, 2127551054, 1867256254, 2124011688, 2582046417, 2556561158, 636426200, 4285016996, 2674887144, 3549740457, 2181041734, 2570240958, 2699124548, 4042917572, 2255489046, 3682010328, 2742005017, 2480846780, 2601626443, 2986491088, 1817857720, 498792707, 1018489859, 3431897810, 1895404124, 1402068388, 3232013476, 3509026475, 764452199, 2497697877, 3382703796, 3805243776, 3493145741, 1227230505, 3212009853, 2147029258, 1881725354, 4293465415, 1559514890, 1001797564, 3955923556, 2087724716, 2081465862, 983355717, 100258315, 1793504923, 1805437041, 564016566, 1777719855, 1868287956, 3898455962, 2098849516, 1330153604, 3286491375, 1980200177, 2904126070, 3989408102, 830817543, 2588226722, 284208306, 3241766793, 250182596, 1345321590, 2280407798, 766970252, 2254844786, 3826666781, 607807438, 3460451781, 1348036367, 3791091923, 1975402020, 

In [4]:
ampa_scl = 1.
nmda_e = 0.
nmda_rs = 0.
nmda_fs = 0.
nmda_scl_e = 0.45
nmda_scl_rs = 0.1
nmda_scl_fs = 0.1

gaba = 0
con_t = 0
con_p = 0
con_r = 0
con_if = 0
con_ir = 0
con_i_all = 0
i_del = 0
noise_wt = 0.016
W_scl = 0
decay_gaba = 5.

In [5]:
neuron_model = """
dv/dt = (v - V_L)/tau_m + I_syn/(1000000*uF) : volt
I_syn = I_AMPA + I_NMDA + I_GABA : ampere
I_AMPA : ampere
I_NMDA : ampere
I_GABA : ampere
"""

neuron_groups = {
    'PC': brian2.NeuronGroup(
        N_e, neuron_model,
        threshold='v > -52*mV',
        reset = 'v = -59*mV',
        namespace=dict(
            V_L=-70*mV,
            tau_m=20*ms,
        )
    ),
    'RSI': brian2.NeuronGroup(
        N_ir, neuron_model,
        threshold='v > -52*mV',
        reset = 'v = -59*mV',
        namespace=dict(
            V_L=-70*mV,
            tau_m=20*ms,
        )
    ),
    'FSI': brian2.NeuronGroup(
        N_if, neuron_model,
        threshold='v > -52*mV',
        reset = 'v = -59*mV',
        namespace=dict(
            V_L=-70*mV,
            tau_m=10*ms,
        )
    ),
}

In [6]:
synapse_AMPA_model = """
dg_AMPA_r/dt = -g_AMPA_r/tau_AMPA_r : siemens (clock-driven)
dg_AMPA_d/dt = -g_AMPA_d/tau_AMPA_d : siemens (clock-driven)
g_AMPA = (g_AMPA_r - g_AMPA_d)/norm_AMPA : siemens
I_AMPA_post = -scl_AMPA*g_AMPA*(v_pre - V_E) : ampere (summed)
"""
synapse_AMPA_onpre = """
g_AMPA_r += w
g_AMPA_d += w
"""

synapse_NMDA_model = """
dg_NMDA_r/dt = -g_NMDA_r/tau_NMDA_r : siemens (clock-driven)
dg_NMDA_d/dt = -g_NMDA_d/tau_NMDA_d : siemens (clock-driven)
g_NMDA = (g_NMDA_r - g_NMDA_d)/norm_NMDA : siemens
I_NMDA_post = -scl_NMDA*g_NMDA*(v_pre - V_E)/(1 + exp(-0.062*v_pre/3.57)) : ampere (summed)
"""
synapse_NMDA_onpre = """
g_NMDA_r += w
g_NMDA_d += w
"""

synapse_GABA_model = """
dg_GABA_r/dt = -g_GABA_r/tau_GABA_r : siemens (clock-driven)
dg_GABA_d/dt = -g_GABA_d/tau_GABA_d : siemens (clock-driven)
g_GABA = (g_GABA_r - g_GABA_d)/norm_GABA : siemens
I_GABA_post = -scl_GABA*g_GABA*(v_pre - V_E) : ampere (summed)
"""
synapse_GABA_onpre = """
g_GABA_r += w
g_GABA_d += w
"""

In [7]:
S = brian2.Synapses(
    neuron_groups['PC'], target=neuron_groups['PC'], model=synapse_AMPA_model, on_pre=synapse_AMPA_onpre,
    namespace=dict(
        tau_AMPA_r=0.5*ms,
        tau_AMPA_d=2.*ms,
        norm_AMPA=0.472689,
        scl_AMPA=1.,
        V_E=0.*mV,
        w=825*nS,
    )
)
S.connect(p=0.1, condition='i!=j')
S = brian2.Synapses(
    neuron_groups['PC'], target=neuron_groups['RSI'], model=synapse_AMPA_model, on_pre=synapse_AMPA_onpre,
    namespace=dict(
        tau_AMPA_r=0.5*ms,
        tau_AMPA_d=2.*ms,
        norm_AMPA=0.472689,
        scl_AMPA=1.,
        V_E=0.*mV,
        w=660*nS,
    )
)
S.connect(p=0.4, condition='i!=j')
S = brian2.Synapses(
    neuron_groups['PC'], target=neuron_groups['FSI'], model=synapse_AMPA_model, on_pre=synapse_AMPA_onpre,
    namespace=dict(
        tau_AMPA_r=0.5*ms,
        tau_AMPA_d=2.*ms,
        norm_AMPA=0.472689,
        scl_AMPA=1.,
        V_E=0.*mV,
        w=1567.5*nS,
    )
)
S.connect(p=0.4, condition='i!=j')

In [8]:
S = brian2.Synapses(
    neuron_groups['RSI'], target=neuron_groups['PC'], model=synapse_NMDA_model, on_pre=synapse_NMDA_onpre,
    namespace=dict(
        tau_NMDA_r=2.*ms,
        tau_NMDA_d=100.*ms,
        norm_NMDA=0.904821,
        scl_NMDA=1.,#TODO
        V_E=0.*mV,
        w=825*nS,
    )
)
S.connect(p=0.1, condition='i!=j')
S = brian2.Synapses(
    neuron_groups['RSI'], target=neuron_groups['RSI'], model=synapse_NMDA_model, on_pre=synapse_NMDA_onpre,
    namespace=dict(
        tau_NMDA_r=2.*ms,
        tau_NMDA_d=100.*ms,
        norm_NMDA=0.904821,
        scl_NMDA=1.,#TODO
        V_E=0.*mV,
        w=660*nS,
    )
)
S.connect(p=0.4, condition='i!=j')
S = brian2.Synapses(
    neuron_groups['RSI'], target=neuron_groups['FSI'], model=synapse_NMDA_model, on_pre=synapse_NMDA_onpre,
    namespace=dict(
        tau_NMDA_r=2.*ms,
        tau_NMDA_d=100.*ms,
        norm_NMDA=0.904821,
        scl_NMDA=1.,#TODO
        V_E=0.*mV,
        w=1567.5*nS,
    )
)
S.connect(p=0.4, condition='i!=j')
S = brian2.Synapses(
    neuron_groups['FSI'], target=neuron_groups['PC'], model=synapse_NMDA_model, on_pre=synapse_NMDA_onpre,
    namespace=dict(
        tau_NMDA_r=2.*ms,
        tau_NMDA_d=100.*ms,
        norm_NMDA=0.904821,
        scl_NMDA=1.,#TODO
        V_E=0.*mV,
        w=825*nS,
    )
)
S.connect(p=0.1, condition='i!=j')
S = brian2.Synapses(
    neuron_groups['FSI'], target=neuron_groups['RSI'], model=synapse_NMDA_model, on_pre=synapse_NMDA_onpre,
    namespace=dict(
        tau_NMDA_r=2.*ms,
        tau_NMDA_d=100.*ms,
        norm_NMDA=0.904821,
        scl_NMDA=1.,#TODO
        V_E=0.*mV,
        w=660*nS,
    )
)
S.connect(p=0.4, condition='i!=j')
S = brian2.Synapses(
    neuron_groups['FSI'], target=neuron_groups['FSI'], model=synapse_NMDA_model, on_pre=synapse_NMDA_onpre,
    namespace=dict(
        tau_NMDA_r=2.*ms,
        tau_NMDA_d=100.*ms,
        norm_NMDA=0.904821,
        scl_NMDA=1.,#TODO
        V_E=0.*mV,
        w=1567.5*nS,
    )
)
S.connect(p=0.4, condition='i!=j')

In [9]:
S = brian2.Synapses(
    neuron_groups['RSI'], target=neuron_groups['PC'], model=synapse_GABA_model, on_pre=synapse_GABA_onpre,
    namespace=dict(
        tau_GABA_r=0.5*ms,
        tau_GABA_d=5.*ms,
        norm_GABA=0.697016,
        scl_GABA=1.,#TODO
        V_E=0.*mV,
        w=660*nS,
    )
)
S.connect(p=0.5, condition='i!=j')
S = brian2.Synapses(
    neuron_groups['RSI'], target=neuron_groups['RSI'], model=synapse_GABA_model, on_pre=synapse_GABA_onpre,
    namespace=dict(
        tau_GABA_r=0.5*ms,
        tau_GABA_d=5.*ms,
        norm_GABA=0.697016,
        scl_GABA=1.,#TODO
        V_E=0.*mV,
        w=660*nS,
    )
)
S.connect(p=0.15, condition='i!=j')
S = brian2.Synapses(
    neuron_groups['RSI'], target=neuron_groups['FSI'], model=synapse_GABA_model, on_pre=synapse_GABA_onpre,
    namespace=dict(
        tau_GABA_r=0.5*ms,
        tau_GABA_d=5.*ms,
        norm_GABA=0.697016,
        scl_GABA=1.,#TODO
        V_E=0.*mV,
        w=660*nS,
    )
)
S.connect(p=0.5, condition='i!=j')
S = brian2.Synapses(
    neuron_groups['FSI'], target=neuron_groups['PC'], model=synapse_GABA_model, on_pre=synapse_GABA_onpre,
    namespace=dict(
        tau_GABA_r=0.5*ms,
        tau_GABA_d=5.*ms,
        norm_GABA=0.697016,
        scl_GABA=1.,#TODO
        V_E=0.*mV,
        w=1567.5*nS,
    )
)
S.connect(p=0.5, condition='i!=j')
S = brian2.Synapses(
    neuron_groups['FSI'], target=neuron_groups['RSI'], model=synapse_GABA_model, on_pre=synapse_GABA_onpre,
    namespace=dict(
        tau_GABA_r=0.5*ms,
        tau_GABA_d=5.*ms,
        norm_GABA=0.697016,
        scl_GABA=1.,#TODO
        V_E=0.*mV,
        w=825*nS,
    )
)
S.connect(p=0.35, condition='i!=j')
S = brian2.Synapses(
    neuron_groups['FSI'], target=neuron_groups['FSI'], model=synapse_GABA_model, on_pre=synapse_GABA_onpre,
    namespace=dict(
        tau_GABA_r=0.5*ms,
        tau_GABA_d=5.*ms,
        norm_GABA=0.697016,
        scl_GABA=1.,#TODO
        V_E=0.*mV,
        w=825*nS,
    )
)
S.connect(p=0.6, condition='i!=j')

In [ ]:
P = brian2.PoissonGroup(100, 100*Hz)
S = brian2.Synapses(P, neuron_groups['PC'], on_pre=)